In [ ]:
%qtconsole

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.signal as signal
from scipy.fftpack import fft, ifft, fftfreq
from configparser import ConfigParser

import sys
sys.path.insert(0,'..')
from levitate.nonlinear import hammersteinModel as hammerstein

In [ ]:
folder = '../Measurements/Modulated noise/'
config = ConfigParser()
config.read(folder + 'settings.ini')
fs = float(config['General']['fs'])
nSamples=int(float(config['General']['blocklength']))
resamplings=int(float(config['General']['resamplings']))

fnames = ['F'+str(i+1) for i in range(resamplings)]
#timedata={}
timedata = pd.concat([
    pd.read_csv(folder + fnames[i] + '.txt',delimiter='\t')
    for i in range(resamplings)], axis=1, keys=fnames,names=['File','Channel'])
#for i in range(resamplings):
#    timedata[str(i+1)] = pd.read_csv(folder+'/'+str(i+1)+'.txt',delimiter='\t')
#timedata=pd.Panel(timedata)

In [ ]:
frequencydata=timedata.apply(fft)/nSamples/3.55e-3
db=lambda x: 20*np.log10(np.abs(x)/20e-6)
lpf = signal.iirfilter(12,20e3*2/fs,btype='lowpass')
lowcut = lambda x: signal.lfilter(lpf[0],lpf[1],x) 
filteredinputs = timedata.drop(['Output','Reference'],axis=1,level='Channel').apply(lowcut)
def approx(inp,out):
    model = hammerstein(kernel='gaussian',npoints=nSamples)
    model.approximateNonlinearity(inp,out)
    return model.rawApprox[1]
#approximated = filteredinputs.apply(lambda x: approx( timedata[x.name[0]]['Reference'],x))

